In [5]:
from dotenv import load_dotenv
import sys

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

sys.path.append('..')
from backend.DB.db import Database

In [6]:
load_dotenv()
llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)


In [7]:
db = Database()
reviews = db.query_sql('SELECT title, review_text FROM raw_reviews')
texts = [f'{title}. {text}' for title, text in list(reviews)]
docs = text_splitter.create_documents(texts)

In [8]:
my_activeloop_org_id = "karencfisher" 
my_activeloop_dataset_name = "hotel-reviews"
# my_activeloop_dataset_name = "meditate_text"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
# db.add_documents(docs)

Deep Lake Dataset in hub://karencfisher/hotel-reviews already exists, loading from the storage


-

Dataset(path='hub://karencfisher/hotel-reviews', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (86, 1536)  float32   None   
    id        text      (86, 1)      str     None   
 metadata     json      (86, 1)      str     None   
   text       text      (86, 1)      str     None   


['5781adff-1453-11ee-8f9c-5065f31a3725',
 '5781ae00-1453-11ee-aebc-5065f31a3725',
 '5781ae01-1453-11ee-94d0-5065f31a3725',
 '5781ae02-1453-11ee-ad08-5065f31a3725',
 '5781ae03-1453-11ee-b7b5-5065f31a3725',
 '5781ae04-1453-11ee-8407-5065f31a3725',
 '5781ae05-1453-11ee-8c3a-5065f31a3725',
 '5781ae06-1453-11ee-bab0-5065f31a3725',
 '5781ae07-1453-11ee-a452-5065f31a3725',
 '5781ae08-1453-11ee-8e30-5065f31a3725',
 '5781ae09-1453-11ee-8f45-5065f31a3725',
 '5781ae0a-1453-11ee-b36a-5065f31a3725',
 '5781ae0b-1453-11ee-bdec-5065f31a3725',
 '5781ae0c-1453-11ee-891e-5065f31a3725',
 '5781ae0d-1453-11ee-b367-5065f31a3725',
 '5781ae0e-1453-11ee-a2c0-5065f31a3725',
 '5781ae0f-1453-11ee-9cab-5065f31a3725',
 '5781ae10-1453-11ee-85d6-5065f31a3725',
 '5781ae11-1453-11ee-8526-5065f31a3725',
 '5781ae12-1453-11ee-bf90-5065f31a3725',
 '5781ae13-1453-11ee-947b-5065f31a3725',
 '5781ae14-1453-11ee-b801-5065f31a3725',
 '5781ae15-1453-11ee-a082-5065f31a3725',
 '5781ae16-1453-11ee-bfde-5065f31a3725',
 '5781ae17-1453-

In [9]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever()
)

tools = [
    Tool(name='Retrival QA System',
         func=retrieval_qa.run,
         description="Answering questions")
]

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

In [10]:
response = agent.run("Tell me about reviews that talk about breakfast?")
print(response)



> Entering new  chain...
 I need to find reviews that talk about breakfast.
Action: Retrival QA System
Action Input: reviews that talk about breakfast
Observation:  The reviews mention that the breakfast was a mix of hot and cold items with homemade green chili sauce, and that there were also packaged grab bags available. They also mention that the $14 voucher didn't cover a meal and a coffee, but it was still only a few bucks more.
Thought: I now know the final answer.
Final Answer: Reviews mention that the breakfast was a mix of hot and cold items with homemade green chili sauce, and that there were also packaged grab bags available. They also mention that the $14 voucher didn't cover a meal and a coffee, but it was still only a few bucks more.

> Finished chain.
Reviews mention that the breakfast was a mix of hot and cold items with homemade green chili sauce, and that there were also packaged grab bags available. They also mention that the $14 voucher didn't cover a meal and a co

In [11]:
response = agent.run("What do reviews say about frontdesk service?")
print(response)



> Entering new  chain...
 I should look for reviews about frontdesk service.
Action: Retrival QA System
Action Input: Reviews about frontdesk service
Observation:  The reviews about the front desk service were positive. Customers commented that the staff was friendly and helpful, providing a smooth refund process and complimentary suite upgrade. They also mentioned that the staff was quick to address any issues, such as noise from the road.
Thought: I now know the final answer
Final Answer: The reviews about the front desk service were positive, with customers commenting that the staff was friendly and helpful, providing a smooth refund process and complimentary suite upgrade. They also mentioned that the staff was quick to address any issues, such as noise from the road.

> Finished chain.
The reviews about the front desk service were positive, with customers commenting that the staff was friendly and helpful, providing a smooth refund process and complimentary suite upgrade. They a